In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time


In [34]:
def extract_film_details(url):
    """Trích xuất thông tin chi tiết của một bộ phim từ trang riêng."""
    service = Service()
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Chạy chế độ không hiển thị trình duyệt
    mini_driver = webdriver.Chrome(service=service, options=options)

    film_info_detail = {
        "title": None,
        "genre": None,
        "duration": None,
        "release_date": None,
        "director": None,
        "actors": None,
        "language": None,
        "description": None,
    }

    try:
        mini_driver.get(url)
        time.sleep(2)  # Chờ trang tải

        # Lấy thông tin chi tiết
        try:
            film_info_detail["title"] = mini_driver.find_element(By.CSS_SELECTOR, "div.product-name span.h1").text.strip()
        except:
            pass

        try:
            film_info_detail["genre"] = mini_driver.find_element(By.CSS_SELECTOR, ".movie-genre .std").text.strip()
        except:
            pass

        try:
            film_info_detail["duration"] = mini_driver.find_element(By.CSS_SELECTOR, ".movie-duration .std").text.strip()
        except:
            pass

        try:
            film_info_detail["release_date"] = mini_driver.find_element(By.CSS_SELECTOR, ".movie-release .std").text.strip()
        except:
            pass

        try:
            film_info_detail["director"] = mini_driver.find_element(By.CSS_SELECTOR, ".movie-director .std").text.strip()
        except:
            pass

        try:
            film_info_detail["actors"] = mini_driver.find_element(By.CSS_SELECTOR, ".movie-actors .std").text.strip()
        except:
            pass

        try:
            film_info_detail["language"] = mini_driver.find_element(By.CSS_SELECTOR, ".movie-language .std").text.strip()
        except:
            pass

        try:
            film_info_detail["description"] = mini_driver.find_element(By.CSS_SELECTOR, "meta[name='description']").get_attribute("content").strip()
        except:
            pass

    except Exception as e:
        print(f"Lỗi khi lấy thông tin phim từ {url}: {e}")

    finally:
        mini_driver.quit()

    return film_info_detail

url = "https://www.cgv.vn/default/vietnamese-concert.html"
film_info_detail = extract_film_details(url)
print(film_info_detail)

{'title': 'VIETNAMESE CONCERT FILM: CHÚNG TA LÀ NGƯỜI VIỆT NAM', 'genre': 'Phim tài liệu', 'duration': None, 'release_date': '14/03/2025', 'director': 'Kawaii Tuấn Anh, Hoàng Thùy Linh', 'actors': None, 'language': 'Tiếng Việt - Phụ đề Tiếng Anh', 'description': 'Bộ phim tài liệu về sự kiện âm nhạc kỷ niệm 13 năm ca hát của ca sĩ – nghệ sĩ Hoàng Thùy Linh. Phim đưa công chúng đến gần hơn với ý tưởng, quá trình sáng tạo, rèn luyện của nghệ sĩ và các cộng sự.'}


In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

def extract_film_info(driver, url):
    """Trích xuất danh sách phim từ trang web."""
    film_list = []
    driver.get(url)
    time.sleep(2)  # Chờ trang tải

    # Tìm tất cả phần tử <li> chứa thông tin phim
    films = driver.find_elements(By.CLASS_NAME, "film-lists.item.last")
    for film in films:
        try:
            link_element = film.find_element(By.CSS_SELECTOR, "a.product-image")
            link_url = link_element.get_attribute("href")

            img_element = film.find_element(By.CSS_SELECTOR, "a.product-image img")
            poster = img_element.get_attribute("src")

            film_info_detail = extract_film_details(link_url)

            # Kiểm tra nút mua vé
            ticket_available = bool(film.find_elements(By.CSS_SELECTOR, "button.btn-booking"))

            film_info = {
                "link": link_url,
                "poster": poster,
                "ticket_available": ticket_available,
                "title": film_info_detail.get("title"),
                "genre": film_info_detail.get("genre"),
                "duration": film_info_detail.get("duration"),
                "release_date": film_info_detail.get("release_date"),
                "director": film_info_detail.get("director"),
                "actors": film_info_detail.get("actors"),
                "language": film_info_detail.get("language"),
                "description": film_info_detail.get("description"),
            }

            film_list.append(film_info)

        except Exception as e:
            print(f"Lỗi khi trích xuất phim: {e}")

    return film_list


In [33]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

# Khởi tạo trình điều khiển Chrome
service = Service()
driver = webdriver.Chrome(service=service)

try:
    # Tạo URL cơ bản
    website_url = "https://www.cgv.vn/default/movies/coming-soon-1.html"

    time.sleep(10)  # Chờ 10 giây trước khi thực hiện truy cập

    # Gọi hàm lấy dữ liệu phim (hàm này bạn đã viết trước đó)
    data_list = extract_film_info(driver,website_url)

    # In thông tin từng phim
    for data in data_list:
        print(data)

finally:
    # Đóng trình duyệt sau khi hoàn tất
    driver.quit()



{'link': 'https://www.cgv.vn/default/vietnamese-concert.html', 'poster': 'https://iguov8nhvyobj.vcdn.cloud/media/catalog/product/cache/1/thumbnail/190x260/2e2b8cd282892c71872b9e67d2cb5039/p/o/poster_1__3_4.jpg', 'ticket_available': True, 'title': '', 'genre': 'Phim tài liệu', 'duration': None, 'release_date': '14/03/2025', 'director': 'Kawaii Tuấn Anh, Hoàng Thùy Linh', 'actors': None, 'language': 'Tiếng Việt - Phụ đề Tiếng Anh', 'description': 'Bộ phim tài liệu về sự kiện âm nhạc kỷ niệm 13 năm ca hát của ca sĩ – nghệ sĩ Hoàng Thùy Linh. Phim đưa công chúng đến gần hơn với ý tưởng, quá trình sáng tạo, rèn luyện của nghệ sĩ và các cộng sự.'}
{'link': 'https://www.cgv.vn/default/the-monkey.html', 'poster': 'https://iguov8nhvyobj.vcdn.cloud/media/catalog/product/cache/1/thumbnail/190x260/2e2b8cd282892c71872b9e67d2cb5039/3/5/350x495-themonkey.jpg', 'ticket_available': True, 'title': '', 'genre': 'Kinh Dị', 'duration': None, 'release_date': '14/03/2025', 'director': 'Osgood Perkins', 'acto

In [25]:
import pymongo
from pymongo import MongoClient

def get_database(url: str, database_name: str):
    """
    Kết nối đến cơ sở dữ liệu MongoDB, nếu chưa có thì tự động tạo.

    Args:
        url (str): Địa chỉ kết nối MongoDB.
        database_name (str): Tên cơ sở dữ liệu cần kết nối.

    Returns:
        Database | None: Đối tượng cơ sở dữ liệu MongoDB hoặc None nếu có lỗi.
    """
    try:
        client = MongoClient(url)
        db = client[database_name]

        # Kiểm tra nếu database chưa tồn tại thì tạo collection mặc định
        if database_name not in client.list_database_names():
            db.create_collection("default_collection")
            print(f"Database '{database_name}' đã được tạo.")
        else:
            print(f"Kết nối thành công đến database '{database_name}'.")

        return db
    except Exception as e:
        print(f"Lỗi khi kết nối đến MongoDB: {e}")
        return None



In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pymongo

# Kết nối MongoDB
client = pymongo.MongoClient("mongodb+srv://CgvHub:o@cluster0.ktuns.mongodb.net/")
db = client["cgv_movies"]

# Tạo collection nếu chưa có
if "films" not in db.list_collection_names():
    db.create_collection("films")
    print("✅ Collection 'films' đã được tạo.")

# Chọn collection
properties_col = db["films"]

# Tạo chỉ mục để tránh dữ liệu trùng lặp (title là duy nhất)
properties_col.create_index("title", unique=True)

# URL của các trang phim
NOW_SHOWING_URL = "https://www.cgv.vn/default/movies/now-showing.html"
COMING_SOON_URL = "https://www.cgv.vn/default/movies/coming-soon-1.html"

# Khởi tạo trình điều khiển Chrome
service = Service()
driver = webdriver.Chrome(service=service)

try:
    # Chờ trình duyệt sẵn sàng
    time.sleep(5)

    # Thu thập phim đang chiếu
    time.sleep(5)  # Chờ tải trang hoàn tất
    now_showing_list = extract_film_info(driver,NOW_SHOWING_URL)

    # Thu thập phim sắp chiếu
    time.sleep(5)
    coming_soon_list = extract_film_info(driver,COMING_SOON_URL)

    # Lưu dữ liệu vào MongoDB (tránh lưu trùng lặp)
    for data in now_showing_list + coming_soon_list:
        result = properties_col.update_one(
            {"title": data["title"]},  # Tìm theo tiêu đề phim
            {"$set": data},            # Cập nhật nội dung
            upsert=True                # Nếu không có thì thêm mới
        )

        if result.matched_count == 0:
            print(f"✅ Đã lưu: {data['title']}")
        else:
            print(f"⚠️ Đã cập nhật: {data['title']}")

except Exception as e:
    print(f"❌ Lỗi xảy ra: {e}")

finally:
    # Đóng trình duyệt
    driver.quit()

✅ Đã lưu: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
✅ Đã lưu: None
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
✅ Đã lưu: RIDER: GIAO HÀNG CHO MA
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
⚠️ Đã cập nhật: 
